### Week 2, Day 2

We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [9]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [2]:
from accounts import Account

In [3]:
account = Account.get("Ed")
account

Account(name='ed', balance=10000.0, strategy='', holdings={}, transactions=[], portfolio_value_time_series=[])

In [4]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ed", "balance": 9756.514, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 81.162, "timestamp": "2025-07-09 21:26:54", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-07-09 21:26:54", 9837.514]], "total_portfolio_value": 9837.514, "total_profit_loss": -162.4860000000008}'

In [5]:
account.report()

'{"name": "ed", "balance": 9756.514, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 81.162, "timestamp": "2025-07-09 21:26:54", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-07-09 21:26:54", 9837.514], ["2025-07-09 21:26:58", 9882.514]], "total_portfolio_value": 9882.514, "total_profit_loss": -117.48600000000079}'

In [6]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 81.162,
  'timestamp': '2025-07-09 21:26:54',
  'rationale': 'Because this bookstore website looks promising'}]

### Now we write an MCP server and use it directly!

In [7]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()


In [8]:
mcp_tools

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', '

In [11]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"

In [16]:
oaiclient = AsyncOpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
model = OpenAIChatCompletionsModel(model="llama3.2:1b", openai_client=oaiclient)

async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


{"type":"function","function":".get_balance", "parameters": {"name": "Ed"}} , {"type":"function","function":".get_holdings", "parameters": {"name": "Ed"}}}

### Now let's build our own MCP Client

In [13]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None), Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None), Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', 'ty

In [14]:
agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
result = await Runner.run(agent, request)
display(Markdown(result.final_output))

In [15]:
context = await read_accounts_resource("ed")
print(context)

{"name": "ed", "balance": 9756.514, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 81.162, "timestamp": "2025-07-09 21:26:54", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-07-09 21:26:54", 9837.514], ["2025-07-09 21:26:58", 9882.514], ["2025-07-09 21:30:46", 9924.514]], "total_portfolio_value": 9924.514, "total_profit_loss": -75.48600000000079}


In [ ]:
from accounts import Account
Account.get("ed").report()

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Make your own MCP Server! Make a simple function to return the current Date, and expose it as a tool so that an Agent can tell you today's date.<br/>Harder optional exercise: then make an MCP Client, and use a native OpenAI call (without the Agents SDK) to use your tool via your client.
            </span>
        </td>
    </tr>
</table>